In [1]:
import bertsum_keras as bk

In [2]:
json_path = "/home/laurence/BertSum-master/json_data"

In [3]:
examples = bk.create_train_examples(json_path, "bert-base-uncased", 5, 200, 100, 3, "greedy")

Preprocessing /home/laurence/BertSum-master/json_data
preprocessing /home/laurence/BertSum-master/json_data/.train.0.json


In [4]:
inputs = bk.create_inputs(examples, is_test=False)

/home/laurence/BertSum-master/bertsum_keras.py:306: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dataset_dict[key] = np.array(dataset_dict[key])


In [5]:
bert_model = bk.Bert(bert_name="bert-base-uncased", max_length=512)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [6]:
x = inputs[0]
segs = inputs[2]
mask = inputs[-1]
print(x.shape, segs.shape, mask.shape)

(7, 512) (7, 512) (7, 512)


In [6]:
top_vec = bert_model.model(inputs[0], token_type_ids=inputs[2], attention_mask=inputs[-1])

In [7]:
type(top_vec)

transformers.modeling_tf_outputs.TFBaseModelOutputWithPooling

In [8]:
type(top_vec[0])

tensorflow.python.framework.ops.EagerTensor

In [9]:
top_vec[0]

<tf.Tensor: shape=(7, 512, 768), dtype=float32, numpy=
array([[[-0.6228525 ,  0.47705775, -0.72852355, ..., -0.65170324,
          0.9282633 ,  0.25004095],
        [ 0.08786282, -0.04721766,  0.00968676, ...,  0.3908336 ,
          1.349763  ,  0.14772786],
        [ 0.29204395, -0.68668365, -0.70527875, ..., -0.1892346 ,
          1.5639086 ,  0.02289766],
        ...,
        [-1.01998   ,  0.9434413 ,  0.5107963 , ..., -0.7512676 ,
          0.6398389 , -0.9635697 ],
        [ 1.1089482 ,  1.1765004 , -0.04959176, ..., -0.5808663 ,
         -0.00567901, -0.33729383],
        [ 0.21653308,  0.01650142,  0.19013762, ...,  0.44655794,
         -0.42004862, -0.4375704 ]],

       [[-1.0520877 ,  0.15514229, -0.5802913 , ..., -0.49859565,
          0.8459695 ,  0.09799159],
        [ 0.08785596, -0.26873925, -0.6590721 , ..., -0.27494165,
          1.0898298 , -0.7622468 ],
        [-0.78238606, -0.01504785, -1.0313606 , ..., -0.7440619 ,
          0.749732  , -0.04465654],
        ...,

In [11]:
import torch
from pytorch_pretrained_bert import BertModel, BertConfig

import torch.nn as nn
class Bert(nn.Module):
    def __init__(self, load_pretrained_bert):
        super(Bert, self).__init__()
        if(load_pretrained_bert):
            self.model = BertModel.from_pretrained('bert-base-uncased')
        else:
            pass

    def forward(self, x, segs, mask):
        encoded_layers, _ = self.model(x, segs, attention_mask =mask)
        top_vec = encoded_layers[-1]
        return top_vec

In [12]:
torch_bert = Bert(load_pretrained_bert=True)

In [13]:
inputs_ids = torch.tensor(inputs[0])
segs = torch.tensor(inputs[2])
mask = torch.tensor(inputs[-1])
print(inputs_ids.shape, segs.shape, mask.shape)

torch.Size([7, 512]) torch.Size([7, 512]) torch.Size([7, 512])


In [ ]:
encoded_layers, _ = torch_bert.model(inputs_ids, segs, attention_mask=mask)

In [30]:
type(encoded_layers)

list

In [31]:
type(encoded_layers[0])

torch.Tensor

In [40]:
encoded_layers[-1].shape

torch.Size([7, 512, 768])

In [42]:
top_vec[0].shape

TensorShape([7, 512, 768])

In [43]:
encoded_layers[-1]

tensor([[[-0.3854,  0.5001, -0.5426,  ..., -0.8447,  0.8990,  0.3143],
         [-0.0523,  0.3153, -0.2423,  ...,  0.6283,  1.3094,  0.1210],
         [ 0.2542, -0.8722, -0.6893,  ..., -0.0851,  1.2619, -0.1486],
         ...,
         [-1.1014,  0.8092,  0.5658,  ..., -0.8050,  0.6798, -0.9332],
         [ 1.0245,  1.1942, -0.3408,  ..., -0.2273,  0.1260, -0.6028],
         [ 0.2259,  0.3663, -0.3053,  ...,  0.2607, -0.3363, -0.5347]],

        [[-0.4633,  0.1499, -0.6180,  ..., -0.4502,  0.9284, -0.0198],
         [ 0.1534, -0.4577, -0.7635,  ..., -0.2426,  1.2170, -0.6065],
         [-0.5922, -0.2721, -0.7822,  ..., -0.7947,  0.7395, -0.1093],
         ...,
         [ 0.7261, -0.5626, -0.0772,  ..., -0.2024, -0.4263, -0.5072],
         [ 0.3820,  0.1813, -0.5868,  ...,  0.0969, -0.5504, -0.2844],
         [-0.8082, -0.3379, -0.5275,  ...,  0.0494,  0.0983, -0.4806]],

        [[-0.5030,  0.1813, -0.4087,  ...,  0.1253,  0.8911, -0.2616],
         [ 0.0560,  0.0440, -0.2835,  ...,  0

In [46]:
top_vec[0]

<tf.Tensor: shape=(7, 512, 768), dtype=float32, numpy=
array([[[-0.6228525 ,  0.47705775, -0.72852355, ..., -0.65170324,
          0.9282633 ,  0.25004095],
        [ 0.08786282, -0.04721766,  0.00968676, ...,  0.3908336 ,
          1.349763  ,  0.14772786],
        [ 0.29204395, -0.68668365, -0.70527875, ..., -0.1892346 ,
          1.5639086 ,  0.02289766],
        ...,
        [-1.01998   ,  0.9434413 ,  0.5107963 , ..., -0.7512676 ,
          0.6398389 , -0.9635697 ],
        [ 1.1089482 ,  1.1765004 , -0.04959176, ..., -0.5808663 ,
         -0.00567901, -0.33729383],
        [ 0.21653308,  0.01650142,  0.19013762, ...,  0.44655794,
         -0.42004862, -0.4375704 ]],

       [[-1.0520877 ,  0.15514229, -0.5802913 , ..., -0.49859565,
          0.8459695 ,  0.09799159],
        [ 0.08785596, -0.26873925, -0.6590721 , ..., -0.27494165,
          1.0898298 , -0.7622468 ],
        [-0.78238606, -0.01504785, -1.0313606 , ..., -0.7440619 ,
          0.749732  , -0.04465654],
        ...,